In [11]:
import json
import pickle
import cv2

# pip install import-ipynb
import import_ipynb
from stereo_camera_calibration import *

In [12]:
def read_keypoints(path):
    
    with open(path) as input_file:
        all_data = json.load(input_file)['captures'][2:]
    left, right = [], []
    left_frame_keypoints = []
    right_frame_keypoints = []
    
    for i in range(len(all_data)):
        if i % 2 == 0:
            left.append(all_data[i])
        else:
            right.append(all_data[i])
            
    for i in range(len(left)):
        left_frame_keypoints.append(get_frame_points(left[i]['annotations'][3]['values'][0]['keypoints']))
    for i in range(len(right)):
        right_frame_keypoints.append(get_frame_points(right[i]['annotations'][3]['values'][0]['keypoints']))
        
    print("find {} frames from left camera.".format(len(left_frame_keypoints)))
    for i in range(len(left_frame_keypoints)):
        print("find {} key points in left frame {}.".format(len(left_frame_keypoints[i]), i))
    print("=====================================================")
    print("find {} frames from right camera.".format(len(right_frame_keypoints)))
    for i in range(len(right_frame_keypoints)):
        print("find {} key points in right frame {}.".format(len(right_frame_keypoints[i]), i))
    
    keypoint_data = {"left": left_frame_keypoints, "right": right_frame_keypoints}
    return keypoint_data

def get_frame_points(frame):
    result = []
    for i in range(len(frame)):
        if frame[i]['x'] != 0 and frame[i]['y'] != 0:
            result.append((frame[i]['x'], frame[i]['y']))
    return result

In [13]:
def read_cal_model(path):
    with open(path, "rb") as input_file:
        cal_data = pickle.load(input_file)
    return cal_data

In [14]:
def triangulate(cal_data, keypoint_data):
    R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(
        cal_data.camera_model["M1"],
        cal_data.camera_model["dist1"],
        cal_data.camera_model["M2"],
        cal_data.camera_model["dist2"],
        (1024, 1024),
        cal_data.camera_model["R"],
        cal_data.camera_model["T"]
    )
    
#     print("P1", P1)
#     print("P2", P2)
    left_frame_keypoints = keypoint_data["left"]
    right_frame_keypoints = keypoint_data["right"]
    
    # iterate on number of frames
    for i in range(len(left_frame_keypoints)):
        # iterate on each pair of key point per frame
        for j in range(len(left_frame_keypoints[i])):
            triang_point = cv2.triangulatePoints(P1, P2, left_frame_keypoints[i][j], right_frame_keypoints[i][j])
            # print(triang_point)
            euclidian = triang_point / triang_point[3]
            euclidian = np.asarray(euclidian, dtype=np.float32)
            print(f"Euclidian Co-ordinate: X:{euclidian[0]} Y:{euclidian[1]} Z:{euclidian[2]}")

In [15]:
if __name__ == "__main__":
    keypoint_data = read_keypoints("./test_frames/TestJsons/captures_000.json")
    cal_data = read_cal_model("./camera_model.pickle")
    triangulate(cal_data, keypoint_data)

find 2 frames from left camera.
find 10 key points in left frame 0.
find 10 key points in left frame 1.
find 2 frames from right camera.
find 10 key points in right frame 0.
find 10 key points in right frame 1.
Euclidian Co-ordinate: X:[-3.2038932] Y:[-1.0778527] Z:[1.3063514]
Euclidian Co-ordinate: X:[-3.203107] Y:[-1.0729501] Z:[1.3061768]
Euclidian Co-ordinate: X:[-3.2025158] Y:[-1.0692524] Z:[1.3060455]
Euclidian Co-ordinate: X:[-3.2019224] Y:[-1.0655583] Z:[1.3059136]
Euclidian Co-ordinate: X:[-3.2013617] Y:[-1.0620515] Z:[1.3057891]
Euclidian Co-ordinate: X:[-3.199001] Y:[-1.0766696] Z:[1.3043331]
Euclidian Co-ordinate: X:[-3.198213] Y:[-1.0717503] Z:[1.3041581]
Euclidian Co-ordinate: X:[-3.197622] Y:[-1.0680609] Z:[1.3040268]
Euclidian Co-ordinate: X:[-3.1970313] Y:[-1.0643713] Z:[1.3038956]
Euclidian Co-ordinate: X:[-3.1964672] Y:[-1.0608515] Z:[1.3037703]
Euclidian Co-ordinate: X:[-3.6786668] Y:[-0.80090505] Z:[1.5149431]
Euclidian Co-ordinate: X:[-3.674433] Y:[-0.8039632] Z:[